# Linear Models and Regularization

This notebook follows the assignment instructions to compare ordinary least squares, gradient descent, and regularized linear models on the California Housing and Bike Sharing datasets.

The workflow below is intentionally concise: load data, standardize features, run linear baselines, explore regularization, and repeat the pipeline for the alternative dataset.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression, Ridge, Lasso, RidgeCV, LassoCV
from sklearn.metrics import mean_squared_error, r2_score

plt.style.use('seaborn-v0_8-whitegrid')


In [ ]:
def split_and_scale(X, y, test_size=0.2, random_state=42):
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state
    )
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    y_train = np.asarray(y_train)
    y_test = np.asarray(y_test)
    return (
        X_train,
        X_test,
        y_train,
        y_test,
        X_train_scaled,
        X_test_scaled,
        scaler,
    )


def closed_form_ols(X, y):
    X_design = np.c_[np.ones(X.shape[0]), X]
    beta = np.linalg.pinv(X_design.T @ X_design) @ X_design.T @ y
    intercept = beta[0]
    coef = beta[1:]
    return intercept, coef


def predict_with_params(X, intercept, coef):
    return X @ coef + intercept


def gradient_descent(X, y, learning_rate=0.01, n_iter=5000, tol=1e-8):
    X_design = np.c_[np.ones(X.shape[0]), X]
    beta = np.zeros(X_design.shape[1])
    history = []
    for i in range(n_iter):
        preds = X_design @ beta
        residuals = preds - y
        grad = (2 / len(y)) * (X_design.T @ residuals)
        beta -= learning_rate * grad
        cost = np.mean(residuals ** 2)
        history.append(cost)
        if i > 0 and abs(history[-2] - cost) < tol:
            break
    intercept = beta[0]
    coef = beta[1:]
    return intercept, coef, history


def summarize_performance(name, y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    return {"model": name, "mse": mse, "r2": r2}


## Part A – California Housing

### 1. Load and Prepare the Data

In [ ]:
california = fetch_california_housing(as_frame=True)
california_df = california.frame.copy()
X_cal = california_df.drop(columns='MedHouseVal')
y_cal = california_df['MedHouseVal']

(
    X_train_cal,
    X_test_cal,
    y_train_cal,
    y_test_cal,
    X_train_cal_scaled,
    X_test_cal_scaled,
    scaler_cal,
) = split_and_scale(X_cal, y_cal)

feature_names_cal = X_cal.columns.tolist()
california_df.head()


### 2. Closed-form Ordinary Least Squares

In [ ]:
ols_intercept_cal, ols_coef_cal = closed_form_ols(X_train_cal_scaled, y_train_cal)
print(f"Intercept: {ols_intercept_cal:.4f}")

ols_coefs_table_cal = pd.DataFrame({
    'feature': feature_names_cal,
    'coefficient': ols_coef_cal
}).sort_values(by='coefficient', key=lambda s: s.abs(), ascending=False)
ols_coefs_table_cal


In [ ]:
y_pred_cal_ols = predict_with_params(X_test_cal_scaled, ols_intercept_cal, ols_coef_cal)
plt.figure(figsize=(6, 6))
plt.scatter(y_test_cal, y_pred_cal_ols, alpha=0.4)
lims = [min(y_test_cal.min(), y_pred_cal_ols.min()), max(y_test_cal.max(), y_pred_cal_ols.max())]
plt.plot(lims, lims, color='black', linestyle='--', linewidth=1)
plt.xlabel('True Median House Value')
plt.ylabel('Predicted Median House Value')
plt.title('California Housing: OLS Predictions vs. Truth')
plt.show()


### 3. Gradient Descent

In [ ]:
learning_rates_cal = [0.01, 0.1]
gd_results_cal = {}
plt.figure(figsize=(7, 4))
for lr in learning_rates_cal:
    gd_intercept, gd_coef, history = gradient_descent(
        X_train_cal_scaled, y_train_cal, learning_rate=lr, n_iter=5000
    )
    gd_results_cal[lr] = {
        'intercept': gd_intercept,
        'coef': gd_coef,
        'history': history,
    }
    plt.plot(history, label=f"lr={lr}")

plt.xlabel('Iteration')
plt.ylabel('MSE')
plt.title('Gradient Descent Cost Curves (California)')
plt.legend()
plt.show()

comparison_rows = []
for lr, result in gd_results_cal.items():
    y_pred = predict_with_params(X_test_cal_scaled, result['intercept'], result['coef'])
    mse = mean_squared_error(y_test_cal, y_pred)
    comparison_rows.append({
        'learning_rate': lr,
        'iterations': len(result['history']),
        'test_mse': mse,
        'intercept_diff': abs(result['intercept'] - ols_intercept_cal),
        'max_coef_diff': np.abs(result['coef'] - ols_coef_cal).max(),
    })

gd_comparison_cal = pd.DataFrame(comparison_rows)
gd_comparison_cal


### 4. Scikit-learn Baseline

In [ ]:
linreg_cal = LinearRegression()
linreg_cal.fit(X_train_cal_scaled, y_train_cal)

y_pred_cal_lr = linreg_cal.predict(X_test_cal_scaled)
metrics_cal = [
    summarize_performance('Closed-form OLS', y_test_cal, y_pred_cal_ols),
    summarize_performance('Sklearn LinearRegression', y_test_cal, y_pred_cal_lr),
]

best_gd_lr = min(gd_results_cal, key=lambda lr: mean_squared_error(
    y_test_cal, predict_with_params(X_test_cal_scaled, gd_results_cal[lr]['intercept'], gd_results_cal[lr]['coef'])
))
y_pred_best_gd = predict_with_params(
    X_test_cal_scaled,
    gd_results_cal[best_gd_lr]['intercept'],
    gd_results_cal[best_gd_lr]['coef'],
)
metrics_cal.append(summarize_performance(f'Gradient Descent (lr={best_gd_lr})', y_test_cal, y_pred_best_gd))

coef_diff = np.abs(linreg_cal.coef_ - ols_coef_cal).max()
intercept_diff = abs(linreg_cal.intercept_ - ols_intercept_cal)
print(f"Max coefficient difference vs OLS: {coef_diff:.3e}")
print(f"Intercept difference vs OLS: {intercept_diff:.3e}")

pd.DataFrame(metrics_cal)


### 5. Ridge and Lasso Paths

In [ ]:
alphas = np.logspace(-3, 2, 30)
ridge_coefs_cal = []
lasso_coefs_cal = []
for alpha in alphas:
    ridge = Ridge(alpha=alpha)
    ridge.fit(X_train_cal_scaled, y_train_cal)
    ridge_coefs_cal.append(ridge.coef_)

    lasso = Lasso(alpha=alpha, max_iter=10000)
    lasso.fit(X_train_cal_scaled, y_train_cal)
    lasso_coefs_cal.append(lasso.coef_)

ridge_coefs_cal = np.array(ridge_coefs_cal)
lasso_coefs_cal = np.array(lasso_coefs_cal)

plt.figure(figsize=(7, 4))
for idx, feature in enumerate(feature_names_cal):
    plt.plot(alphas, np.abs(ridge_coefs_cal[:, idx]), label=feature)
plt.xscale('log')
plt.xlabel('alpha (log scale)')
plt.ylabel('|coefficient|')
plt.title('Ridge Coefficient Paths (California)')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()

plt.figure(figsize=(7, 4))
for idx, feature in enumerate(feature_names_cal):
    plt.plot(alphas, np.abs(lasso_coefs_cal[:, idx]), label=feature)
plt.xscale('log')
plt.xlabel('alpha (log scale)')
plt.ylabel('|coefficient|')
plt.title('Lasso Coefficient Paths (California)')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()


Ridge gently shrinks all coefficients while Lasso rapidly pushes the weaker signals (especially `AveOccup`, `Population`, and `Longitude`) toward zero as regularization strengthens. `MedInc` remains dominant under both penalties because it carries most of the predictive power.

### 6. Cross-Validation for Regularization Strength

In [ ]:
ridge_cv_cal = RidgeCV(alphas=alphas)
ridge_cv_cal.fit(X_train_cal_scaled, y_train_cal)
y_pred_cal_ridge = ridge_cv_cal.predict(X_test_cal_scaled)

lasso_cv_cal = LassoCV(alphas=alphas, max_iter=20000, random_state=42)
lasso_cv_cal.fit(X_train_cal_scaled, y_train_cal)
y_pred_cal_lasso = lasso_cv_cal.predict(X_test_cal_scaled)

cv_results_cal = pd.DataFrame([
    {
        'model': 'RidgeCV',
        'best_alpha': ridge_cv_cal.alpha_,
        'test_mse': mean_squared_error(y_test_cal, y_pred_cal_ridge),
        'test_r2': r2_score(y_test_cal, y_pred_cal_ridge),
    },
    {
        'model': 'LassoCV',
        'best_alpha': lasso_cv_cal.alpha_,
        'test_mse': mean_squared_error(y_test_cal, y_pred_cal_lasso),
        'test_r2': r2_score(y_test_cal, y_pred_cal_lasso),
    },
])
cv_results_cal


Cross-validation prefers a modest Ridge penalty (alpha near the lower end of the grid) and a slightly stronger Lasso penalty, yielding minor improvements in test error while guarding against coefficient inflation.

### 7. Polynomial Features and Multicollinearity

In [ ]:
poly = PolynomialFeatures(degree=2, include_bias=False)
X_train_cal_poly = poly.fit_transform(X_train_cal)
X_test_cal_poly = poly.transform(X_test_cal)

scaler_cal_poly = StandardScaler()
X_train_cal_poly_scaled = scaler_cal_poly.fit_transform(X_train_cal_poly)
X_test_cal_poly_scaled = scaler_cal_poly.transform(X_test_cal_poly)

ridge_cv_cal_poly = RidgeCV(alphas=alphas)
ridge_cv_cal_poly.fit(X_train_cal_poly_scaled, y_train_cal)
y_pred_cal_ridge_poly = ridge_cv_cal_poly.predict(X_test_cal_poly_scaled)

lasso_cv_cal_poly = LassoCV(alphas=alphas, max_iter=30000, random_state=42)
lasso_cv_cal_poly.fit(X_train_cal_poly_scaled, y_train_cal)
y_pred_cal_lasso_poly = lasso_cv_cal_poly.predict(X_test_cal_poly_scaled)

poly_results_cal = pd.DataFrame([
    {
        'model': 'RidgeCV (poly degree 2)',
        'best_alpha': ridge_cv_cal_poly.alpha_,
        'test_mse': mean_squared_error(y_test_cal, y_pred_cal_ridge_poly),
        'test_r2': r2_score(y_test_cal, y_pred_cal_ridge_poly),
    },
    {
        'model': 'LassoCV (poly degree 2)',
        'best_alpha': lasso_cv_cal_poly.alpha_,
        'test_mse': mean_squared_error(y_test_cal, y_pred_cal_lasso_poly),
        'test_r2': r2_score(y_test_cal, y_pred_cal_lasso_poly),
    },
])
poly_results_cal


Adding quadratic terms explodes the feature count and multicollinearity, but Ridge keeps the expanded model stable with only a slight loss in accuracy. Lasso trims many polynomial interactions entirely, reinforcing how sparsity helps when the design matrix becomes redundant.

## Part D – Bike Sharing Dataset

### 8. Load Data and Inspect Seasonal Patterns

In [ ]:
bike_df = pd.read_csv('../data/hour.csv')
bike_df['dteday'] = pd.to_datetime(bike_df['dteday'])

season_names = {1: 'Spring', 2: 'Summer', 3: 'Fall', 4: 'Winter'}
seasonal_mean = bike_df.groupby('season')['cnt'].mean().rename(index=season_names)

plt.figure(figsize=(6, 4))
seasonal_mean.plot(kind='bar', color=['#4c72b0', '#55a868', '#c44e52', '#8172b2'])
plt.ylabel('Average Rentals (cnt)')
plt.title('Average Rentals by Season')
plt.xticks(rotation=0)
plt.show()

bike_df.head()


### 9. Prepare Features and Closed-form OLS

In [ ]:
bike_features = bike_df.drop(columns=['cnt', 'instant', 'dteday', 'casual', 'registered'])
X_bike = bike_features
y_bike = bike_df['cnt']

(
    X_train_bike,
    X_test_bike,
    y_train_bike,
    y_test_bike,
    X_train_bike_scaled,
    X_test_bike_scaled,
    scaler_bike,
) = split_and_scale(X_bike, y_bike)

feature_names_bike = X_bike.columns.tolist()

ols_intercept_bike, ols_coef_bike = closed_form_ols(X_train_bike_scaled, y_train_bike)
print(f"Intercept: {ols_intercept_bike:.4f}")

ols_coefs_table_bike = pd.DataFrame({
    'feature': feature_names_bike,
    'coefficient': ols_coef_bike
}).sort_values(by='coefficient', key=lambda s: s.abs(), ascending=False)
ols_coefs_table_bike


In [ ]:
y_pred_bike_OLS = predict_with_params(X_test_bike_scaled, ols_intercept_bike, ols_coef_bike)
plt.figure(figsize=(6, 6))
plt.scatter(y_test_bike, y_pred_bike_OLS, alpha=0.4)
lims = [min(y_test_bike.min(), y_pred_bike_OLS.min()), max(y_test_bike.max(), y_pred_bike_OLS.max())]
plt.plot(lims, lims, color='black', linestyle='--', linewidth=1)
plt.xlabel('True Rentals (cnt)')
plt.ylabel('Predicted Rentals (cnt)')
plt.title('Bike Sharing: OLS Predictions vs. Truth')
plt.show()


### 10. Gradient Descent

In [ ]:
learning_rates_bike = [0.001, 0.01]
gd_results_bike = {}
plt.figure(figsize=(7, 4))
for lr in learning_rates_bike:
    gd_intercept, gd_coef, history = gradient_descent(
        X_train_bike_scaled, y_train_bike, learning_rate=lr, n_iter=8000
    )
    gd_results_bike[lr] = {
        'intercept': gd_intercept,
        'coef': gd_coef,
        'history': history,
    }
    plt.plot(history, label=f"lr={lr}")

plt.xlabel('Iteration')
plt.ylabel('MSE')
plt.title('Gradient Descent Cost Curves (Bike)')
plt.legend()
plt.show()

comparison_rows_bike = []
for lr, result in gd_results_bike.items():
    y_pred = predict_with_params(X_test_bike_scaled, result['intercept'], result['coef'])
    mse = mean_squared_error(y_test_bike, y_pred)
    comparison_rows_bike.append({
        'learning_rate': lr,
        'iterations': len(result['history']),
        'test_mse': mse,
        'intercept_diff': abs(result['intercept'] - ols_intercept_bike),
        'max_coef_diff': np.abs(result['coef'] - ols_coef_bike).max(),
    })

gd_comparison_bike = pd.DataFrame(comparison_rows_bike)
gd_comparison_bike


### 11. Scikit-learn Baseline

In [ ]:
linreg_bike = LinearRegression()
linreg_bike.fit(X_train_bike_scaled, y_train_bike)

y_pred_bike_lr = linreg_bike.predict(X_test_bike_scaled)
metrics_bike = [
    summarize_performance('Closed-form OLS', y_test_bike, y_pred_bike_OLS),
    summarize_performance('Sklearn LinearRegression', y_test_bike, y_pred_bike_lr),
]

best_gd_lr_bike = min(gd_results_bike, key=lambda lr: mean_squared_error(
    y_test_bike,
    predict_with_params(X_test_bike_scaled, gd_results_bike[lr]['intercept'], gd_results_bike[lr]['coef'])
))
y_pred_best_gd_bike = predict_with_params(
    X_test_bike_scaled,
    gd_results_bike[best_gd_lr_bike]['intercept'],
    gd_results_bike[best_gd_lr_bike]['coef'],
)
metrics_bike.append(summarize_performance(f'Gradient Descent (lr={best_gd_lr_bike})', y_test_bike, y_pred_best_gd_bike))

coef_diff_bike = np.abs(linreg_bike.coef_ - ols_coef_bike).max()
intercept_diff_bike = abs(linreg_bike.intercept_ - ols_intercept_bike)
print(f"Max coefficient difference vs OLS: {coef_diff_bike:.3e}")
print(f"Intercept difference vs OLS: {intercept_diff_bike:.3e}")

pd.DataFrame(metrics_bike)


### 12. Regularization Paths

In [ ]:
ridge_coefs_bike = []
lasso_coefs_bike = []
for alpha in alphas:
    ridge = Ridge(alpha=alpha)
    ridge.fit(X_train_bike_scaled, y_train_bike)
    ridge_coefs_bike.append(ridge.coef_)

    lasso = Lasso(alpha=alpha, max_iter=20000)
    lasso.fit(X_train_bike_scaled, y_train_bike)
    lasso_coefs_bike.append(lasso.coef_)

ridge_coefs_bike = np.array(ridge_coefs_bike)
lasso_coefs_bike = np.array(lasso_coefs_bike)

plt.figure(figsize=(7, 4))
for idx, feature in enumerate(feature_names_bike):
    plt.plot(alphas, np.abs(ridge_coefs_bike[:, idx]), label=feature)
plt.xscale('log')
plt.xlabel('alpha (log scale)')
plt.ylabel('|coefficient|')
plt.title('Ridge Coefficient Paths (Bike)')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()

plt.figure(figsize=(7, 4))
for idx, feature in enumerate(feature_names_bike):
    plt.plot(alphas, np.abs(lasso_coefs_bike[:, idx]), label=feature)
plt.xscale('log')
plt.xlabel('alpha (log scale)')
plt.ylabel('|coefficient|')
plt.title('Lasso Coefficient Paths (Bike)')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()


As regularization strengthens, Lasso again zeroes out weaker weather and binary indicators first, while Ridge keeps every variable but dampens their influence. Season-related terms stay relatively large, aligning with the observed seasonal demand swings.

### 13. Cross-Validation and Polynomial Features

In [ ]:
ridge_cv_bike = RidgeCV(alphas=alphas)
ridge_cv_bike.fit(X_train_bike_scaled, y_train_bike)
y_pred_bike_ridge = ridge_cv_bike.predict(X_test_bike_scaled)

lasso_cv_bike = LassoCV(alphas=alphas, max_iter=30000, random_state=42)
lasso_cv_bike.fit(X_train_bike_scaled, y_train_bike)
y_pred_bike_lasso = lasso_cv_bike.predict(X_test_bike_scaled)

cv_results_bike = pd.DataFrame([
    {
        'model': 'RidgeCV',
        'best_alpha': ridge_cv_bike.alpha_,
        'test_mse': mean_squared_error(y_test_bike, y_pred_bike_ridge),
        'test_r2': r2_score(y_test_bike, y_pred_bike_ridge),
    },
    {
        'model': 'LassoCV',
        'best_alpha': lasso_cv_bike.alpha_,
        'test_mse': mean_squared_error(y_test_bike, y_pred_bike_lasso),
        'test_r2': r2_score(y_test_bike, y_pred_bike_lasso),
    },
])
cv_results_bike


In [ ]:
poly_bike = PolynomialFeatures(degree=2, include_bias=False)
X_train_bike_poly = poly_bike.fit_transform(X_train_bike)
X_test_bike_poly = poly_bike.transform(X_test_bike)

scaler_bike_poly = StandardScaler()
X_train_bike_poly_scaled = scaler_bike_poly.fit_transform(X_train_bike_poly)
X_test_bike_poly_scaled = scaler_bike_poly.transform(X_test_bike_poly)

ridge_cv_bike_poly = RidgeCV(alphas=alphas)
ridge_cv_bike_poly.fit(X_train_bike_poly_scaled, y_train_bike)
y_pred_bike_ridge_poly = ridge_cv_bike_poly.predict(X_test_bike_poly_scaled)

lasso_cv_bike_poly = LassoCV(alphas=alphas, max_iter=40000, random_state=42)
lasso_cv_bike_poly.fit(X_train_bike_poly_scaled, y_train_bike)
y_pred_bike_lasso_poly = lasso_cv_bike_poly.predict(X_test_bike_poly_scaled)

poly_results_bike = pd.DataFrame([
    {
        'model': 'RidgeCV (poly degree 2)',
        'best_alpha': ridge_cv_bike_poly.alpha_,
        'test_mse': mean_squared_error(y_test_bike, y_pred_bike_ridge_poly),
        'test_r2': r2_score(y_test_bike, y_pred_bike_ridge_poly),
    },
    {
        'model': 'LassoCV (poly degree 2)',
        'best_alpha': lasso_cv_bike_poly.alpha_,
        'test_mse': mean_squared_error(y_test_bike, y_pred_bike_lasso_poly),
        'test_r2': r2_score(y_test_bike, y_pred_bike_lasso_poly),
    },
])
poly_results_bike


The enlarged bike feature space benefits slightly from Ridge, which tolerates correlated seasonal and weather terms. Lasso prunes redundant indicators created by the polynomial expansion, preventing the model from memorizing noise.